In [69]:
import backtrader as bt
from datetime import datetime
import pandas as pd

# Load data from CSV
df = pd.read_csv('./MSFT.csv', index_col='Date', parse_dates=True)
start_date = '2025-01-01'
end_date = '2025-10-01'
df = df[df.index >= start_date]
df = df[df.index < end_date]
pandasdata = bt.feeds.PandasData(dataname=df, open=0, high=1, low=2, close=3, volume=5)
print("Data loaded successfully.")

Data loaded successfully.


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT at the first day start_date. 

2. Hold the 100 shares until end_date without buying additional shares. 

3. Output the return of this strategy.

In [70]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.order = None
        self.bought = False

    def next(self):
        if not self.bought and len(self) == 1:
            self.order = self.buy(size = 100, exectype = bt.Order.Close)
            self.bought = True


In [71]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100_000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value - init_cash) / init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


The one year return of the strategy is: 0.09460000610351563
The cash left is: 57664.99938964844
Current position: 100 shares of MSFT


---

1. Backtest a strategy with backtrader that ALL-IN MSFT from the first day start_date. 

2. Hold all the shares until end_date. 

3. Output the return of this strategy.

In [ ]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.order = None   # Tracking orders
        self.bought = False # Labelling if bought
        
    def next(self):
        if not self.bought and len(self) == 1:
            cash = self.broker.getcash()
            price = self.datas[0].open[0]
            size = int(cash // price)
            try:
                self.order = self.buy(size = size)
                self.bought = True
                print(f"Bought {size} shares at ${price:.2f} with ${cash:.2f} cash")
            except Exception as e:
                print(f"Error placing order: {e}")


In [73]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Bought 235 shares at $425.53 with $100000.00 cash
The one year return of the strategy is: 0.2276445602416992
The cash left is: 1046.2031555175781
Current position: 235 shares of MSFT


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT every 30 trading days until out of cash from the first day start_date. 

2. For example, the first buying date is the 1st trading day since start_date, the second buying date is the 31th trading day, the third is 61 trading date, etc. 

3. Hold all the shares until end_date. 

4. Output the return of this strategy.

In [74]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.order = None
        self.last_buy_day = -31
        self.buy_interval = 30
        
    def next(self):
        days_since_last_buy = len(self) - 1 - self.last_buy_day
        
        if days_since_last_buy >= self.buy_interval:
            
            cash = self.broker.getcash()
            price = self.datas[0].close[0]
            cost = 100 * price
            
            self.last_buy_day = len(self) - 1
            
            if cash >= cost:
                self.order = self.buy(size = 100)
                print(f"Day {len(self)}: Bought 100 shares at ${price:.2f}, Cost: ${cost:.2f}, Cash left: ${cash-cost:.2f}")
            else:
                print(f"Day {len(self)}: Not enough cash to buy 100 shares. Need ${cost:.2f}, Have ${cash:.2f}")


In [75]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Day 1: Bought 100 shares at $418.58, Cost: $41858.00, Cash left: $58142.00
Day 31: Bought 100 shares at $409.64, Cost: $40964.00, Cash left: $16928.00
Day 61: Not enough cash to buy 100 shares. Need $38219.00, Have $17104.00
Day 91: Not enough cash to buy 100 shares. Need $45294.00, Have $17104.00
Day 121: Not enough cash to buy 100 shares. Need $49594.00, Have $17104.00
Day 151: Not enough cash to buy 100 shares. Need $52177.00, Have $17104.00
Day 181: Not enough cash to buy 100 shares. Need $50923.00, Have $17104.00
The one year return of the strategy is: 0.2069400329589844
The cash left is: 17104.000854492188
Current position: 200 shares of MSFT


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT when the loss of a day is over 2\% (compare the close price with open price in one day)

In [76]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.order = None
        
    def next(self):
        if self.order:
            return
        
        open_price = self.datas[0].open[0]
        close_price = self.datas[0].close[0]
        
        daily_return = (close_price - open_price) / open_price
        
        if daily_return <= -0.02:
            cash = self.broker.getcash()
            cost = 100 * close_price
            
            if cash >= cost:
                self.order = self.buy(size = 100, exectype = bt.Order.Close)
                print(f"Day {len(self)}: Daily loss {daily_return*100:.2f}% (>2%), bought 100 shares at ${close_price:.2f}")
            else:
                print(f"Day {len(self)}: Not enough cash to buy 100 shares. Need ${cost:.2f}, Have ${cash:.2f}")
    
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.order = None


In [77]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Day 34: Daily loss -2.19% (>2%), bought 100 shares at $408.21
Day 38: Daily loss -2.18% (>2%), bought 100 shares at $392.53
Day 40: Not enough cash to buy 100 shares. Need $38849.00, Have $19901.00
Day 59: Not enough cash to buy 100 shares. Need $37880.00, Have $19901.00
Day 66: Not enough cash to buy 100 shares. Need $35456.00, Have $19901.00
Day 72: Not enough cash to buy 100 shares. Need $37161.00, Have $19901.00
Day 144: Not enough cash to buy 100 shares. Need $53350.00, Have $19901.00
Day 145: Not enough cash to buy 100 shares. Need $52411.00, Have $19901.00
Day 153: Not enough cash to buy 100 shares. Need $52058.00, Have $19901.00
Day 169: Not enough cash to buy 100 shares. Need $49500.00, Have $19901.00
The one year return of the strategy is: 0.2349100341796875
The cash left is: 19901.0009765625
Current position: 200 shares of MSFT
